In [1]:
#  INSTALL IPYTHON-SQL:
# !pip install ipython-sql --quiet


# IMPORT NECESSARY LIBRARIES:
import pandas as pd
import sqlite3


# DEFINE PATHS:
BANK_CHURNERS_PATH = './data/csv/bank_churners.csv'
CLIENT_INFO_PATH = './data/csv/basic_client_info.csv'
CHURN_DATA_PATH = './data/csv/enriched_churn_data.csv'
DB_PATH = './data/db/churn.db'

NOTES & REFERENCES: 
https://holypython.com/python-sqlite-tutorial/python-magic-for-sql/?expand_article=1 

```
%%sql result << 
QUERY
```
for multi-line query 

```
% sql result = QUERY
```
for single line query

Reading the data into a pandas a dataframe: 

In [2]:
bank_churners = pd.read_csv(BANK_CHURNERS_PATH)
client_info = pd.read_csv(CLIENT_INFO_PATH)
churn_data= pd.read_csv(CHURN_DATA_PATH)

In [3]:
display(bank_churners.head(2),
        client_info.head(2),
        churn_data.head(2)
        )

,clientnum,attrition_flag,dependent_count,card_category,months_on_book,months_inactive_12_mon,contacts_count_12_mon,credit_limit,avg_open_to_buy,avg_utilization_ratio
0,806160108,Existing Customer,1,Blue,56,2,3,3193.0,676.0,0.788
1,804424383,Existing Customer,1,Blue,56,3,2,10215.0,9205.0,0.099


,clientnum,customer_age,gender,education_level,marital_status,income_category
0,708082083,45,F,High School,Married,Less than $40K
1,708083283,58,M,Unknown,Single,$40K - $60K


,clientnum,total_relationship_count,total_revolving_bal,total_amt_chng_q4_q1,total_trans_amt,total_ct_chng_q4_q1,total_trans_ct
0,828343083,3,1793,0.803,3646,0.659,68
1,828298908,4,2035,0.613,1770,0.741,47


In [4]:
database = sqlite3.connect(DB_PATH)
bank_churners.to_sql('bank_churners', database)
client_info.to_sql('client_info', database)
churn_data.to_sql('churn_data', database)

10127

In [5]:
%load_ext sql
%sql sqlite:///data/db/churn.db 

In [15]:
%%sql

SELECT * 
from client_info
where ROW_NUM < 5

 * sqlite:///data/db/churn.db
(sqlite3.OperationalError) no such column: ROW_NUM
[SQL: SELECT * 
from client_info
where ROW_NUM < 5]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
result = _ 
result

DESCRIBE THE TABLE:

In [ ]:
%%sql 

SELECT *
FROM customers;
PRAGMA table_info(customers);

/*data cleaning*/
-- checking for misssing values

In [ ]:
%%sql

SELECT *
FROM customers
WHERE CLIENTNUM is NULL OR Attrition_Flag IS NULL OR Customer_Age IS NULL OR Gender IS NULL OR Dependent_count is NULL OR Education_Level IS NULL OR Marital_Status IS Null;

-- there are no missing values

/*unique Customers
There's a total of 10127 customers*/

In [ ]:
%%sql 

SELECT 
DISTINCT CLIENTNUM
FROM customers
LIMIT 5;

In [ ]:
%%sql 

SELECT 
concat(round(COUNT(CASE WHEN Attrition_Flag = '1' THEN Clientnum END) / COUNT(Clientnum),2) * 100,'%') AS churn_rate,
concat(round(COUNT(CASE WHEN Attrition_Flag = '0' THEN Clientnum END) / COUNT(Clientnum),2) * 100,'%') AS retention_rate
FROM customers;

In [ ]:
%%sql 

SELECT DISTINCT Attrition_Flag
FROM customers;

/*Column Manipulation
transforming the customer_age column into bins using the case statement*/

In [ ]:
%%sql

SELECT 
COUNT(Attrition_FLag) AS existing_customer_count
FROM customers
WHERE Attrition_Flag ='Existing Customer';

-- there are 8500 existing customers

In [ ]:
%%sql

SELECT 
COUNT(Attrition_Flag) AS churned_customer_count
FROM customers
WHERE Attrition_Flag ='Attrited Customer';

-- there are 1627 churned customers indicating high imbalance

In [ ]:
%%sql

SELECT
Attrition_FLag,
CASE 
WHEN Attrition_FLag ='Attrited Customer' THEN 1 
ELSE 0 END AS attrited_customer
FROM customers
GROUP BY Attrition_Flag;

In [ ]:
%%sql

UPDATE customers
SET Attrition_Flag=CASE 
WHEN Attrition_FLag ='Attrited Customer' THEN 1 
ELSE 0 END;
SELECT 
Attrition_Flag
FROM customers
GROUP BY Attrition_Flag;

In [ ]:
%%sql

SELECT 
round(COUNT(CASE WHEN Attrition_Flag = 1 THEN CLIENTNUM END) / COUNT(CLIENTNUM), 2) * 100  AS churn_rate,
round(COUNT(CASE WHEN Attrition_Flag = 0 THEN CLIENTNUM END) / COUNT(CLIENTNUM), 2) * 100  AS retention_rate
FROM customers;

-- customer Age

In [ ]:
%%sql 

SELECT 
MAX(Customer_Age) AS max_age,
MIN(Customer_Age) AS min_age
FROM customers;

-- the eldest customer is 73 while the youngest is 26

In [ ]:
%%sql 

SELECT 
    Customer_Age,
    SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
    SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
    ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) * 1.0 / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2) || '%' AS churn_rate,
    ROUND((SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) * 1.0 / (SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END))) * 100, 2) || '%' AS retention_rate
FROM customers
GROUP BY Customer_Age
ORDER BY Customer_Age ASC;

In [ ]:
%%sql

SELECT 
    Gender,
    SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
    SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
    CAST(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) * 1.0 / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2) AS TEXT) || '%' AS churn_rate
FROM customers
GROUP BY Gender
ORDER BY Gender ASC;


In [ ]:
%%sql 

SELECT 
Dependent_count,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2) || '%' AS churn_rate
FROM customers
GROUP BY Dependent_count
ORDER BY Dependent_count ASC; 

In [ ]:
%%sql 

SELECT 
Education_Level,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate
FROM customers
GROUP BY Education_Level
ORDER BY Education_Level ASC;

In [ ]:
%%sql

SELECT 
Marital_Status,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate
FROM customers
GROUP BY Marital_Status
ORDER BY Marital_Status ASC;

In [ ]:
%%sql

SELECT 
Income_Category,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) -SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS difference,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate
FROM customers
GROUP BY Income_Category
ORDER BY Income_Category ASC;

In [ ]:
%%sql

SELECT 
Card_Category,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) -SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS difference,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate
FROM customers
GROUP BY Card_Category
ORDER BY Card_Category ASC;

In [ ]:
%%sql

SELECT 
Months_on_book,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) -SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS difference,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END))) * 100, 2), '%') AS retention_rate
FROM customers
GROUP BY Months_on_book
ORDER BY Months_on_book ASC;

In [ ]:
%%sql

SELECT 
Total_Relationship_Count,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) -SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS difference,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END))) * 100, 2), '%') AS retention_rate
FROM customers
GROUP BY Total_Relationship_Count
ORDER BY Total_Relationship_Count ASC;

In [ ]:
%%sql

SELECT 
Months_Inactive_12_mon,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END))) * 100, 2), '%') AS retention_rate
FROM customers
GROUP BY Months_Inactive_12_mon
ORDER BY Months_Inactive_12_mon ASC;


In [ ]:
%%sql

SELECT 
Contacts_Count_12_mon,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count,
CONCAT(ROUND((SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / (SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) + SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END))) * 100, 2), '%') AS churn_rate
FROM customers
GROUP BY Contacts_Count_12_mon
ORDER BY Contacts_Count_12_mon ASC;

In [ ]:
%%sql

SELECT 
Attrition_Flag,
SUM(Total_Trans_Amt) AS total_transaction_amount,
SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) AS churned_customer_count,
SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) AS existing_customer_count
FROM customers
GROUP BY Attrition_Flag
ORDER BY SUM(Total_Trans_Amt) DESC;

In [ ]:
%%sql

SELECT 
CONCAT(ROUND(SUM(CASE WHEN Attrition_Flag = '1' THEN 1 ELSE 0 END) / COUNT(*) * 100, 2), '%') AS churned_customer_percentage,
CONCAT(ROUND(SUM(CASE WHEN Attrition_Flag = '0' THEN 1 ELSE 0 END) / COUNT(*) * 100, 2), '%') AS existing_customer_percentage
FROM customers;

PROJECT 2: This is an analysis of a Bank credit card customers in SQL. The objective is to know the demographics of churned customers, attrition based on card category and month on book.

/*Age Group Distribution of Attrited Customers*/

In [ ]:
%%sql

Select Case When Customer_Age<20 Then "0-20"
       When Customer_Age Between 20 And 30 Then "20-30" When Customer_Age Between 30 And 40 Then "30-40"
       When Customer_Age Between 40 And 50 Then "40-50" When Customer_Age Between 50 And 60 Then "50-60"
       When Customer_Age Between 60 And 70 Then "60-70" When Customer_Age Between 70 And 80 Then "70-80"
       When Customer_Age>80 Then "Above 80" End As Age_Range, Count(*) 
From customers 
Where Attrition_Flag = "Attrited Customer"
Group by Age_Range 
Order by Age_Range;

/*Proportion of Male and Female in Attrited and Existing Customers*/

In [ ]:
%%sql

Select SUM(If(Gender="M",1,"NULL")) As MaleExistingCustomers,
	   SUM(If(Gender="F",1,"NULL")) As FemaleExistingCustomers
From customers
Where Attrition_Flag = "Existing Customer";
Select SUM(If(Gender="M",1,"NULL")) As MaleAttritedCustomers,
	   SUM(If
	       (Gender = "F",1,"NULL")) As FemaleAttritedCustomers
From customers
Where Attrition_Flag = "Attrited Customer";  

/*Distribution of Attrited and Existing Customers based on Dependent Count*/

In [ ]:
%%sql

Select Dependent_count, Count(*)
From customers
Where Attrition_Flag = "Existing Customer" 
Group by Dependent_count
Order by Dependent_count;
Select Dependent_count, Count(*)
From customers
Where Attrition_Flag = "Attrited Customer" 
Group by Dependent_count
Order by Dependent_count;


/*Education Level of Existing and Attrited Customers*/

In [ ]:
%%sql

Select Education_level, Count(*)
From customers
Where Attrition_Flag= "Existing Customer" 
Group by Education_Level
Order by Count(*);
Select Education_level, Count(*)
From customers
Where Attrition_Flag= "Attrited Customer" 
Group by Education_Level
Order by Count(*);


/*Marital Status of Existing and Attrited Customers*/

In [ ]:
%%sql

Select Marital_Status, Count(*)
From customers
Where Attrition_Flag= "Existing Customer" 
Group by Marital_Status
Order by Count(*);
Select Marital_Status, Count(*)
From customers
Where Attrition_Flag= "Attrited Customer" 
Group by Marital_Status
Order by Count(*);

/*Distribution of Existing and Attrited Customers Based on Card Category*/

In [ ]:
%%sql

Select Card_Category, Count(*)
From customers
Where Attrition_Flag= "Existing Customer" 
Group by Card_Category
Order by Count(*);
Select Card_Category, Count(*)
From customers
Where Attrition_Flag= "Attrited Customer" 
Group by Card_Category
Order by Count(*);

/*Attrited Customers based on Months on Book*/

In [ ]:
%%sql

Select Case When Months_on_book<20 Then "0-20"
       When Months_on_book Between 20 And 30 Then "20-30" When Months_on_book Between 30 And 40 Then "30-40"
       When Months_on_book Between 40 And 50 Then "40-50" When Months_on_book Between 50 And 60 Then "50-60"
       When Customer_Age>60 Then "Above 60" End As MonthsonBook_Range, Count(*) 
From customers 
Where Attrition_Flag = "Attrited Customer" 
Group by MonthsonBook_Range 
Order by MonthsonBook_Range;


/*Inactive Months of Existing and Attrited Customers*/

In [ ]:
%%sql

Select Months_Inactive_12_mon, Count(*)
From customers
Where Attrition_Flag = "Attrited Customer"
Group by Months_Inactive_12_mon
Order by Count(*);
Select Months_Inactive_12_mon, Count(*)
From customers
Where Attrition_Flag = "Existing Customer"
Group by Months_Inactive_12_mon
Order by Count(*);


/*Average Utilization of Existing and Attrited Customer*/

In [ ]:
%%sql

Select Attrition_Flag, Round(Avg(Avg_Utilization_Ratio)*100,2) As AvgUtilization
From customers
Where Attrition_Flag="Attrited Customer";
Select Attrition_Flag, Round(Avg(Avg_Utilization_Ratio)*100,2) As AvgUtilization
From customers
Where Attrition_Flag="Existing Customer";